In [2]:
import time
import collections
import uuid

import pandas

import amostra
import amostra.client.local_commands as acc



try.nsls2.bnl.gov

nsls-ii.github.io







In [3]:
owner = "arkilic"
project = "amostra_demo"
beamline_id = "dbx"

In [5]:
# Connection Pool
from amostra.client.commands import ContainerReference, SampleReference
container_client = acc.ContainerReference(host="xf23id-ca.cs.nsls2.local", port=7772)
sample_client = acc.SampleReference(host="xf23id-ca.cs.nsls2.local", port=7772)

In [6]:
def insert_plate(plate_info):
    plate = container_client.create(**plate_info)
    return plate

def insert_sample_list(samples):
    res = []
    for s in samples:
        res.append(sample_client.create(s))
    return res

In [7]:
def generate_plate(owner, project, beamline_id, kind, name, barcode, content):
    plate_info = {    
            "owner": owner,
            "project": project,
            "beamline_id": beamline_id,
            "kind": kind,
            "name": name,
            "barcode": barcode,
            "content": list(content)
    }

    return insert_plate(plate_info)

In [8]:
def generate_samples_96wp(project, beamline_id, owner, name):
    samples = collections.deque()
    for i in range(96):
        sample_info = {
            "project": project,
            "beamline_id": beamline_id,
            "owner": owner,
            "name": name+" {}".format(i+1),
            "position": {"x": int(i % 8), "y": int(i/8)},
            "concentration": 0.001,
            "volume": 10,
            "temperature": 21.4    
        }
        samples.append(sample_info)
    uids = insert_sample_list(samples)
    return uids



In [9]:
def find_plate_by_barcode(owner, project, beamline_id, barcode):
    samples = collections.deque()
    plate_info = list(container_client.find(owner=owner, project=project, 
                                            beamline_id=beamline_id, barcode=barcode))[0]
    for s_uid in plate_info['content']:
        samples.append(next(sample_client.find(uid=s_uid)))
    
    plate_info['content'] = list(samples)
    return plate_info

In [10]:
number_of_plates = 5

for i in range(number_of_plates):
    samples = generate_samples_96wp(project, beamline_id, owner, "Test Sample")
    plate1 = generate_plate(owner, project, beamline_id, "96wp", "Plate {}".format(i+1), "{}".format(i+1).zfill(13), samples)

In [14]:
an_inserted_uid = samples[0]['uid']
next(sample_client.find(uid=an_inserted_uid))

{'container': 'NULL',
 'name': {'beamline_id': 'dbx',
  'concentration': 0.001,
  'name': 'Test Sample 1',
  'owner': 'arkilic',
  'position': {'x': 0, 'y': 0},
  'project': 'amostra_demo',
  'temperature': 21.4,
  'volume': 10},
 'time': 1475605068.833802,
 'uid': 'a9da8f2d-8e32-46cc-893a-f8a1fc31253c'}

In [16]:
next(sample_client.find(**{'name.position.x': 0}))

{'container': 'NULL',
 'name': {'beamline_id': 'dbx',
  'concentration': 0.001,
  'name': 'Test Sample 1',
  'owner': 'arkilic',
  'position': {'x': 0, 'y': 0},
  'project': 'amostra_demo',
  'temperature': 21.4,
  'volume': 10},
 'time': 1474905928.305002,
 'uid': 'fd5496f0-8d78-47b7-a137-2b06234928e7'}

In [12]:
print(an_inserted_uid)

a9da8f2d-8e32-46cc-893a-f8a1fc31253c


In [ ]:
def import_plate_from_excel(fname, owner, project, beamline_id, plate_kind):
    samples = collections.deque()
    excel_data = pandas.read_excel(fname,header=1)
    
    for line in excel_data.iterrows():
        if line[0] == 0:
            name = line[1][0]
            barcode = str(int(line[1][1])).zfill(13)
            plate_info = {    
                "owner": owner,
                "project": project,
                "beamline_id": beamline_id,
                "kind": plate_kind,
                "name": name,
                "barcode": barcode,
            }
        s_x = line[1][2]
        s_y = line[1][3]
        s_name = line[1][4]
        s_shortname = line[1][5]
        s_conc = line[1][6]
        s_volume = line[1][7]
        s_temperature = line[1][8]
        sample_info = {
                "project": project,
                "beamline_id": beamline_id,
                "owner": owner,
                "name": s_name,
                "short_name": s_shortname,
                "position": {"x": s_x, "y": s_y},
                "concentration": s_conc,
                "volume": s_volume,
                "temperature": s_temperature        
            }
        samples.append(sample_info)
        
    return plate_info, samples